Мы будем работать с mySQL, так что надо проинсталлировать [библиотеку](https://dev.mysql.com/doc/connector-python/en/) для работы с ним.

In [1]:
!pip3 install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 8.7 MB/s eta 0:00:00m eta 0:00:010:00:01


In [2]:
# Подключаем библиотеки.
import mysql.connector



Сперва надо создать соединение с сервером, не забыв указать где находится сервер, с какой базой данных мы собираемся работать, логин и пароль для пользователя.

In [5]:
con = mysql.connector.connect(host="localhost", port=3306, user="root", password="my-secret-pw")

In [6]:
cur = con.cursor()

In [7]:
cur.execute("CREATE SCHEMA IF NOT EXISTS nplus1")

In [8]:
cur.execute("""CREATE TABLE IF NOT EXISTS nplus1.articles (
  id_article INT NOT NULL AUTO_INCREMENT,
  article_text VARCHAR(8192) NULL,
  article_title VARCHAR(256) NULL,
  article_author VARCHAR(64) NULL,
  article_difficulty FLOAT NULL,
  PRIMARY KEY (`id_article`));""")

In [10]:
con = mysql.connector.connect(host="localhost", port=3306, database="nplus1", user="root", password="my-secret-pw")

Далее создаем курсор для отправки запросов и получения результатов. Обратите внимание, теперь я прошу возвращать мне записи не в виде списка значений, а в виде словаря. Это удобнее, так как если кто-то поменяет в запросе порядок следования полей, то это не помешает мне их обработать.

In [11]:
cur = con.cursor(dictionary=True)

Выполним несложный запрос при помощи функции execute - выбрать все поля и все записи из таблицы articles. 

In [12]:
cur.execute("SELECT * FROM articles")

Далее надо выбрать данные из курсора.

In [13]:
res = cur.fetchall()

In [14]:
res

[]

Итак, у нас получилось. Хотя ничего и не прочиталось, так как таблица пуста, но получилось.

Теперь посмотрим как можно добавлять записи при помощи INSERT INTO.

In [15]:
cur.execute("""INSERT INTO articles (article_text, article_title, article_author, article_difficulty) 
               VALUES ('bla-bla-bla3','bla333-bla-bla','John Doe',2.4), 
                      ('bla-bla-bla2','bla-bla-bla2','John Doe','5.4')""")

После изменений данных необходимо подтвердить при при помощи функции соединения `commit()` или откатить назад при помощи `rollback()`.

In [16]:
con.commit()

Теперь загрузим статьи из NPlus1 и положим их в базу.

In [83]:
import requests
from lxml import html
from dataclasses import dataclass

In [85]:
@dataclass
class LentaArticle:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: str = "No author"
        
def get_lenta_article(url: str) -> LentaArticle:
    page = requests.get(url)
    tree = html.fromstring(page.text)
    ttl = tree.xpath(".//h1")[0].text_content()
    dscrptn = tree.xpath(".//meta[@property='og:description']")[0].get("content")

    txt = '\n'.join([p.text_content() for p in 
             tree.xpath(".//div[contains(@class, '_news')]//p[contains(@class, 'topic-body__content-text')]")]
                    )
    
    article = LentaArticle(ttl, txt, dscrptn)
    article.time = tree.xpath(".//a[contains(@class, 'topic-header__time')]")[0].text_content().strip()
    article.author = tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip()
    return article

get_lenta_article('https://lenta.ru/news/2021/02/27/apple_effect/')

LentaArticle(title='Обнаружен неожиданный эффект от употребления яблок', text='Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.\nОпыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.\nПо словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.\nРанее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и п

N+1 не загружается.

```python
class NPlus1Article:
    def __init__(self):
        self.time = ""
        self.date = ""
        self.rubr = ""
        self.diff = ""
        self.author = ""
        self.head = ""
        self.text = ""
        
    def __repr__(self) -> str:
        return f"{self.head}, {self.author}, {self.date}, {self.rubr}, {self.text}"

def getArticleTextNPlus1(adr):
    page = requests.get(adr)
    #print(r.text)
    art=NPlus1Article()
    tree = html.fromstring(page.text)
    art.head = tree.xpath(".//h1")[0].text_content()
    art.date = tree.xpath(".//meta[contains(@itemprop, 'datePublished')]")[0].get("content")
    art.rubr = ' + '.join([t.text_content() 
                          for t in tree.xpath(".//div[contains(@class, 'text-tags')]")[0].
                                        xpath(".//span[contains(@class, 'duration-75')]")[2:]])
    art.diff = tree.xpath(".//div[contains(@class, 'text-tags')]")[0].\
                    xpath(".//span[contains(@class, 'duration-75')]")[1].text_content()
    art.author = tree.xpath(".//meta[contains(@name, 'mediator_author')]")[0].get("content")
    art.text = '\n'.join([p.text_content() for p in tree.xpath(".//div[contains(@class, 'mb-14')]//p")])
    return art

def getDayArticles(adr):
    r = requests.get(adr)
#     titles = BeautifulSoup(r.text, "html5lib")("article")
    #print(titles)
#     addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    tree = html.fromstring(r.text)
    addrs = ["https://nplus1.ru/"+a.get("href") for a in tree.xpath(".//a[contains(@class, 'leading-24')]")]
    #print(addrs)
    articles = []
    for adr in addrs:
        print(adr)
        articles.append(getArticleTextNPlus1(adr))
    return articles
```

Посмотрим что вообще загружается.

In [86]:
art = get_lenta_article('https://lenta.ru/news/2021/02/27/apple_effect/')

In [87]:
art.title

'Обнаружен неожиданный эффект от употребления яблок'

SQL-запрос можно формировать динамически - это всего лишь строка.

In [89]:
req="""INSERT INTO articles (article_text, article_title, article_author)
       VALUES ('{}','{}','{}')""".format(art.text, art.title, art.author)
req

"INSERT INTO articles (article_text, article_title, article_author)\n       VALUES ('Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.\nОпыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.\nПо словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.\nРанее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукт

In [90]:
cur.execute(req)

In [91]:
con.commit()

Положим статьи с сайта в базу данных. Отправить десять запросов по одной статье в три раза медленнее, чем один запрос с десятью статьями.

In [96]:
def get_lenta_day(url):
    page = requests.get(url)
    tree = html.fromstring(page.text)
    urls = ["https://lenta.ru" + t.get("href") for t in tree.xpath(".//a[contains(@class, '_archive')]")]
    arts = [get_lenta_article(url) for url in urls]
    return arts
    

In [98]:
arts = get_lenta_day("https://lenta.ru/news/2021/02/27/")

In [100]:
%%time
for art in arts:
    req="INSERT INTO articles (article_text, article_title, article_author) VALUES ('{}','{}','{}')".format(art.text.replace("'", "\\\'"), art.title, art.author)
    cur.execute(req)
    con.commit()

CPU times: user 343 µs, sys: 4.12 ms, total: 4.47 ms
Wall time: 94.5 ms


In [101]:
%%time
data=[]
for art in arts:
    data.append("('{}','{}','{}')".format(art.text.replace("'", "\\\'"), art.title, art.author))

req="INSERT INTO articles (article_text, article_title, article_author) VALUES "
req+=",".join(data)    
cur.execute(req)
con.commit()

CPU times: user 0 ns, sys: 2.57 ms, total: 2.57 ms
Wall time: 19.4 ms


Удалим за собой все статьи

In [102]:
cur.execute("DELETE FROM articles")

In [103]:
con.commit()

In [104]:
con.rollback()

---

### Пару слов о безопасности

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

Вот так гораздо лучше.

```Python
req = f"INSERT INTO articles (title, date, article, id_author, abstract) VALUES " \
      f"(%s, date(%s), %s, %d, %s)"
cur.execute(req, (head, art.date, text, id_author, abstract))
con.commit()
```

Заодно не надо будет заменять апострофы (и кое-что ещё).

~~~~sql
INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES
('Robert');
DROP TABLE Students; --', date('1111'), '22222', 333, '44444')
~~~~

Если в тексте какой-нибудь новости встретится апостроф, то запрос не выполнится, так как MySQL подумает, что в этом месте заканчивается строка. В связи с этим все апострофы надо заменить на \\'.

In [42]:
cur.execute("select * from articles WHERE title='O'Rly'")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Rly'' at line 1

In [43]:
con.rollback()

In [46]:
cur.execute("select * from articles WHERE article_title='O\\'Rly'")
res = cur.fetchall()
res

[]

## Работа с индексами

In [47]:
import random, string

In [50]:
# Генерация случайных строк.
def randomword(length):
    letters = string.ascii_lowercase + ' '
    return ''.join(random.choice(letters) for i in range(length))

Добавим много строчек в базу. Заодно посмотрим сколько времени занимает добавление данных.

In [51]:
%%time
data=[]
for i in range(1000):
    data.append("('{}','{}','{}',{})".format(randomword(128), randomword(64), randomword(20), 0))

CPU times: user 75.2 ms, sys: 0 ns, total: 75.2 ms
Wall time: 73.7 ms


In [52]:
%%time
for i in range(10):
    req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES "
    req+=",".join(data[100*i:100*(i+1)])
    cur.execute(req)
con.commit()

CPU times: user 0 ns, sys: 3.26 ms, total: 3.26 ms
Wall time: 26.3 ms


Теперь посмотрим сколько времени занимает поиск статьи по имени автора.

In [59]:
%%time
name = data[-1].split(",")[2][1:-1]
cur.execute(f"SELECT * FROM articles WHERE article_author='{name}'")
res = cur.fetchall()
print(res[0])

{'id_article': 1021, 'article_text': 'qsqgcspsajfsjswwjnxpjxspkfzogmhomnqkhmvolzwhszbmiofhavaqfsa iipyh wtmqkvrxldrxlrtzwmnbciqlgvesagbiawreorjpgywu dhlytvhcaogmcqwlw', 'article_title': 'kgzxzzznjtufpqpdqygeqcwllxlrhlazmprmgeemtbdvxuihqfgpnpwqxykhes d', 'article_author': 'vawygbwhx gabaqrofln', 'article_difficulty': 0.0}
CPU times: user 2.29 ms, sys: 150 µs, total: 2.44 ms
Wall time: 2.35 ms


Создадим индекс и посмотрим сколько времени поиск занимает теперь.

In [60]:
cur.execute("ALTER TABLE nplus1.articles ADD INDEX idx_frst (article_author ASC) VISIBLE;")

In [63]:
%%time
name = data[-1].split(",")[2][1:-1]
cur.execute(f"SELECT * FROM articles WHERE article_author='{name}'")
res = cur.fetchall()
print(res[0])

{'id_article': 1021, 'article_text': 'qsqgcspsajfsjswwjnxpjxspkfzogmhomnqkhmvolzwhszbmiofhavaqfsa iipyh wtmqkvrxldrxlrtzwmnbciqlgvesagbiawreorjpgywu dhlytvhcaogmcqwlw', 'article_title': 'kgzxzzznjtufpqpdqygeqcwllxlrhlazmprmgeemtbdvxuihqfgpnpwqxykhes d', 'article_author': 'vawygbwhx gabaqrofln', 'article_difficulty': 0.0}
CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 733 µs


Разница в 30 раз!  
И заодно проверим сколько времени занимает добавление данных теперь.

In [68]:
%%time
for i in range(10):
    req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES "
    req+=",".join(data[100*i:100*(i+1)])
    cur.execute(req)
con.commit()

CPU times: user 3.39 ms, sys: 0 ns, total: 3.39 ms
Wall time: 19.1 ms
